In [4]:
import math
import numpy as np
import kaiwu as kw

 # Number of tasks
N_TASK = 20
# Number of machines
N_MACHINE = 3

seed = 12345
np.random.seed(seed)

# Index sets
SET_TASK = {t for t in range(1, N_TASK + 1)}  # Tasks 1, 2, ..., N_TASK
SET_MACHINE = {m for m in range(1, N_MACHINE + 1)}  # Machines 1, 2, ..., N_MACHINE

# Model data
DURATION = {t: np.random.randint(1, N_TASK + 1) for t in SET_TASK}  # Random task duration between 1 and 20
START = {m: np.random.randint(1, N_MACHINE + 1) for m in SET_MACHINE}  # Random machine idle start time between 1 and 5

# Penalty coefficient for constraints
LAMBDA = 1000

# Output the number of unsatisfied constraints
def get_count_constr_not_met(solution_dict, print_detail=False):
    count_constr_not_met = 0  # Total number of unsatisfied constraints
    for t in SET_TASK:
        constr_value = kw.qubo.get_val(sum(x[(t, m)] for m in SET_MACHINE) - 1, solution_dict)
        if print_detail:
            print('constr_server_capacity', t, constr_value)
        if constr_value != 0:
            count_constr_not_met += 1
    return count_constr_not_met


# Output results
def result_summary(the_sol_dict):
    # Completion time for each machine
    max_complete_time = 0.0
    for m in SET_MACHINE:
        complete_time = kw.qubo.get_val(machine_endtime[m], the_sol_dict)
        if complete_time > max_complete_time:
            max_complete_time = complete_time
    return max_complete_time

In [5]:
# Decision variables
x = {(t, m): kw.qubo.Binary('x[' + str(t - 1) + '][' + str(m - 1) + ']') for t in SET_TASK for m in
        SET_MACHINE}  # Whether task i is assigned to machine j
# QUBO objective function
obj_function = 0.0
# Variance term
obj_function += sum(
    pow(
        START[m] + sum(DURATION[t] * x[(t, m)] for t in SET_TASK) -
        (sum(START[m] for m in SET_MACHINE) + sum(DURATION[t] for t in SET_TASK)) / N_MACHINE,
        2
    )
    for m in SET_MACHINE
) / N_MACHINE
# Constraint term
obj_function += LAMBDA * sum(pow(sum(x[(t, m)] for m in SET_MACHINE) - 1, 2) for t in SET_TASK)
# End time for each machine
machine_endtime = {
    m: START[m] + sum(DURATION[t] * x[(t, m)] for t in SET_TASK)
    for m in SET_MACHINE
}

qubo_model = kw.qubo.QuboModel()
qubo_model.set_objective(obj_function)

# Solve using simulated annealing
max_iter = 40
curr_iter = 0
current_best = math.inf  # Initialize current best solution as infinity
opt_obj = 0  # Desired optimal objective value (adjustable for specific problems)

while (curr_iter < max_iter and current_best > opt_obj):
    print(f'Iteration {curr_iter} starts, current best solution = {current_best}')
    worker = kw.classical.SimulatedAnnealingOptimizer(
        initial_temperature=1000,
        alpha=0.99,
        cutoff_temperature=0.0001,
        iterations_per_t=10)
    solver = kw.solver.SimpleSolver(worker)
    # Get the optimal solution from simulated annealing output
    sol_dict, qubo_val = solver.solve_qubo(qubo_model)

    count_constr_not_met = get_count_constr_not_met(sol_dict, print_detail=False)
    if not count_constr_not_met:
        val_obj = result_summary(sol_dict)
        current_best = min(current_best, val_obj)  # Update current best solution

    curr_iter += 1
print('Optimal solution:', current_best)

Iteration 0 starts, current best solution = inf


ValueError: Please follow the tutorial to generate the license first. If there is no sdk authorization code, please log in to platform.qboson.com for support.